<a href="https://colab.research.google.com/github/doodger/dataScience/blob/development/amazonNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()# SPARK_HOMEfrom pyspark.sql import SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
df = spark.createDataFrame([{"hello": "world"} for x in range(1000)])
df.show(3)

/content/spark-2.4.5-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+-----+
|hello|
+-----+
|world|
|world|
|world|
+-----+
only showing top 3 rows



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#!ls drive/'My Drive'
from pyspark.sql import SQLContext
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
df = spark.read.json("drive/My Drive/Video_Games_5.json")

In [0]:
!ls "drive/My Drive/Video_Games_5.json"

'drive/My Drive/Video_Games_5.json'


In [0]:
df.show()

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0700099867| [8, 12]|    1.0|Installing the ga...| 07 9, 2012|A2HD75EMZR8QLN|                 123|Pay to unlock con...|    1341792000|
|0700099867|  [0, 0]|    4.0|If you like rally...|06 30, 2013|A3UR8NLLY1ZHCX|Alejandro Henao "...|     Good rally game|    1372550400|
|0700099867|  [0, 0]|    1.0|1st shipment rece...|06 28, 2014|A1INA0F5CWW3J4|Amazon Shopper "M...|           Wrong key|    1403913600|
|0700099867| [7, 10]|    3.0|I got this versio...|09 14, 2011|A1DLMTOTHQ4AST|            ampgreen|awesome game, if ...|    1315958400|
|0700099867|  [2, 2]|    4.0|I had Dirt 2 on X...|06 14

In [0]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer,StopWordsRemover
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

tokenizer = Tokenizer(inputCol = "reviewText", outputCol = "words")

tokenized = tokenizer.transform(df)

remover = StopWordsRemover(inputCol="words", outputCol = "filtered")
transformed =  remover.transform(tokenized)


In [0]:
transformed.show()

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|               words|            filtered|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+
|0700099867| [8, 12]|    1.0|Installing the ga...| 07 9, 2012|A2HD75EMZR8QLN|                 123|Pay to unlock con...|    1341792000|[installing, the,...|[installing, game...|
|0700099867|  [0, 0]|    4.0|If you like rally...|06 30, 2013|A3UR8NLLY1ZHCX|Alejandro Henao "...|     Good rally game|    1372550400|[if, you, like, r...|[like, rally, car...|
|0700099867|  [0, 0]|    1.0|1st shipment rece...|06 28, 2014|A1INA0F5CWW3J4|Amazon Shopper "M...|           Wrong 

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, OneHotEncoderEstimator, StringIndexer, CountVectorizer
countVectors = CountVectorizer(inputCol="filtered", outputCol = "countedWords", vocabSize=2000)
model = countVectors.fit(transformed)

In [0]:
newDf = model.transform(transformed)


In [0]:
newDf.show()

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|               words|            filtered|        countedWords|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+
|0700099867| [8, 12]|    1.0|Installing the ga...| 07 9, 2012|A2HD75EMZR8QLN|                 123|Pay to unlock con...|    1341792000|[installing, the,...|[installing, game...|(2000,[1,2,3,5,7,...|
|0700099867|  [0, 0]|    4.0|If you like rally...|06 30, 2013|A3UR8NLLY1ZHCX|Alejandro Henao "...|     Good rally game|    1372550400|[if, you, like, r...|[like, rally, car...|(2000,[1,2,3,12,3...|
|070009986

In [0]:
assembler = VectorAssembler(
    inputCols = ["countedWords"],
    outputCol = "features")

output = assembler.transform(newDf)
output = output.select(col("overall").alias("label"),col("features"))


In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer


sc.setLogLevel("DEBUG")
(trainingData, testData) = output.randomSplit([0.7,0.3])
gbt = GBTRegressor(featuresCol="features")

pipeline = Pipeline(stages=[gbt])
model = pipeline.fit(trainingData)
#10 minutes with no max iteration

In [0]:
predictions = model.transform(testData)

In [0]:
predictions.show(100)

+-----+--------------------+------------------+
|label|            features|        prediction|
+-----+--------------------+------------------+
|  1.0|        (2000,[],[])|4.0564369495986705|
|  1.0|        (2000,[],[])|4.0564369495986705|
|  1.0|(2000,[0,1,2,3,4,...| 2.661828832283631|
|  1.0|(2000,[0,1,2,3,4,...|   3.2967927945398|
|  1.0|(2000,[0,1,2,3,4,...| 3.386578978831921|
|  1.0|(2000,[0,1,2,3,4,...|2.8670581829674546|
|  1.0|(2000,[0,1,2,3,4,...| 4.216047454879624|
|  1.0|(2000,[0,1,2,3,4,...|3.8030895978301302|
|  1.0|(2000,[0,1,2,3,4,...|3.6292877488889976|
|  1.0|(2000,[0,1,2,3,4,...| 4.265135461829432|
|  1.0|(2000,[0,1,2,3,4,...| 3.673055737768444|
|  1.0|(2000,[0,1,2,3,4,...| 4.239083516933368|
|  1.0|(2000,[0,1,2,3,4,...| 2.992138613807334|
|  1.0|(2000,[0,1,2,3,4,...|1.8072936646290485|
|  1.0|(2000,[0,1,2,3,4,...|3.7174162551822194|
|  1.0|(2000,[0,1,2,3,4,...|3.9188649455641515|
|  1.0|(2000,[0,1,2,3,4,...|3.5587055666134035|
|  1.0|(2000,[0,1,2,3,4,...|2.8070797239

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
        labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 1.06739
